## CS585: Natural Language Processing
### LSTM Recurrent Neural Networks

<br><br>
#### Illinois Institute of Technology  
#### Aron Culotta

<br><br><br><br><br>

## Long-term dependencies

(Adapted from [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) and [here](https://arxiv.org/abs/1506.00019))

One of the expected advantages of Recurrent Neural Networks is the ability to capture long-range dependencies.

<br>

For example, consider the language modeling problem. Given a sentence like:

> “I grew up in <u>France</u> ... I speak fluent <u>French</u>.”

A trigram language model would have

$$p(\mathrm{French} \mid \mathrm{speak \: fluent}) \approx p(\mathrm{Spanish} \mid \mathrm{speak \: fluent})$$

But, given the presence of the word "France" earlier in the document, we should instead have that

$$p(\mathrm{French} \mid \mathrm{speak \: fluent}) >> p(\mathrm{Spanish} \mid \mathrm{speak \: fluent})$$

<br><br>
It is unscalable to capture such dependencies in a traditional language model or HMM.

Recall that the Viterbi algorithm takes time $O(S^2T)$, where $T$ is the length of the sequence and $S$ is the number of states.

A second-order Markov model conditions on the prior 2 states:

$p(q_i \mid q_{i-2} q_{i-1})$

- must consider 4 combinations of previous states (assuming binary state variables)


A third-order Markov model conditions on the prior 3 states:

$p(q_i \mid q_{i-3} q_{i-2} q_{i-1})$

- must consider 8 combinations of previous states (assuming binary state variables)

- $\rightarrow$ Viterbi complexity is exponential in the order of the model.
- Not to mention the data sparsity issue of fitting higher order probabilities $p(q_i \mid q_{i-3} q_{i-2} q_{i-1})$




## How do Recurrent Neural Networks encode long-range dependencies?

- Like a first-order HMM, Recurrent NNs only depend on the current input and the prior hidden state.
- However, the hidden state at time $t$ can contain information from an arbitrarily long context window
- How?
  - Given a hidden layer of $n$ binary nodes, how many distinct states can it encode?
    - $2^n$
  - Of course, our hidden nodes are in $\mathbf{R}$, we can represent even more states.
  
Thus, while the <u>expressive power</u> of the network grows exponentially with the number of hidden nodes, the complexity of <u>inference and training</u> grows at most quadratically.

![long](figs/long.png)

E.g., in the "French" example, if $x_1=\mathrm{France}$, and $x_{t+1} = \mathrm{French}$, our hope is that the hidden layer will retain information about "France" from $h_1$ up to $h_{t+1}$, influencing the prediction at time $t+1$.
<br>


## Vanishing / Exploding gradients

While the above explanation holds in theory, in practice there is an important limitation.

![vanish](figs/vanish.png)

- Recall that the weights are duplicated (shared) across time steps
  - e.g., $W_{hh}[i,j]$ is the transition weight from hidden node $i$ to hidden node $j$
- If the weight along a recurrent edge is less than 1, then the contribution of input at time $t$ on the hidden layer at time $t+m$ will decrease with $m$.
  - E.g., for three steps: $W_{hh}[i,j] * W_{hh}[i,j] * W_{hh}[i,j]$
- Conversely, if the weight is greater than 1, the contribution will increase over time.

<br>
** This contribution increase/decrease varies <u>exponentially</u> with m (the time gap)! **
  - E.g., for three steps, it's $W_{hh}[i,j]^3$
  
  
One fix was the hack in the code from last class:

```python
for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
```

But, doing so hampers the ability of the network to learn long-range dependencies.
  - The size of the clip parameters affects the memory size of the network.

## Long short-term memory (LSTM)

An alternative solution to the problem is to change the structure of the network to explicitly allow it to remember/forget prior state information.

Where does the name come from?
- **Long-term memory**: The parameters of the model, which are shared across time steps, are analogous to long-term memory.
- **Short-term memory**: The hidden layers represent short-term memory, since they can change as new input is read.


The idea of LSTMs is to introduce a <u>memory cell</u>:
  - a state that stores the information from prior steps
  - by altering the transition function, we can ensure that the gradient does not explode/vanish over time.
  
  
Consider a traditional recurrent neural network:

![rnn](figs/rnn.png)

An LSTM makes several changes:
![lstm](figs/lstm1.png)

![notation](figs/notation.png)


#### Cell State ($C_t$)

The core idea of LSTM is to introduce a cell state $C_t$ (a memory cell)

![cellstate](figs/cellstate.png)

We'll see below that the cell state $C_t$ is a simpler linear function of the prior cell state $C_{t-1}$, which is what prevents the vanishing gradient problem. 

There are several gates that will decide what information is included in the cell state.

#### Forget Gate Layer

![forget](figs/forget.png)

Note: $W_f \cdot[h_{t-1}, x_t] = W_fh_{t-1} + W_fx_t$

The forget gate outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$.
  - 1 means keep this number
  - 0 means forget this number
  
E.g., in "French" example, 1 means keep the information about "France" in the cell state, so it can be used later to predict the term "French."



#### Input Gate 

![input](figs/input.png)

The input gate layer decides what new information to add to the memory cell.  
- First, a sigmoid layer called the “input gate layer” decides which values we’ll update.
- Next, a tanh layer creates a vector of new candidate values,$\tilde{C}_t$, that could be added to the state.


#### Updating the Cell State

![update](figs/update.png)

To create the new state, we:
- Multiply the old state by $f_t$, forgetting the things we decided to forget earlier.
- Then we add $i_t*\tilde{C}_t$; this is the new candidate values, scaled by how much we decided to update each state value.


#### Output Gate

![output](figs/output.png)

- Similar to the output of recursive NN, but filtered
- First, apply a sigmoid layer to decide which parts of the memory cell state to output.
- Then, apply a tanh layer (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the chosen parts.

<br>

Note: if we just used $f_t=1$ for all $t$, we would just remember all cell states, without forgetting any information.
- This would already solve the diminishing gradient problem.
- The forgetting allows for a sparser, more efficient model.

In [ ]:
"""
Adapted from https://github.com/nicodjimenez/lstm

Learns a LSTM that outputs the value of the first element of the
hidden layer as the prediction.

Note that the output layer is not used in this implementation
"""

import random

import numpy as np
import math

def sigmoid(x): 
    return 1. / (1 + np.exp(-x))

def sigmoid_derivative(values): 
    return values*(1-values)

def tanh_derivative(values): 
    return 1. - values ** 2

# createst uniform random array w/ values in [a,b) and shape args
def rand_arr(a, b, *args): 
    np.random.seed(0)
    return np.random.rand(*args) * (b - a) + a

class LstmParam:
    def __init__(self, mem_cell_ct, x_dim):
        """
        mem_cell_ct: the number of memory cells to create
        x_dim: the dimension of the input vector
        """
        self.mem_cell_ct = mem_cell_ct
        self.x_dim = x_dim
        concat_len = x_dim + mem_cell_ct
        # weight matrices
        # input weights
        self.wg = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        # input gate layer weights
        self.wi = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        # forgetting layer weights
        self.wf = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)
        # output layer weights
        self.wo = rand_arr(-0.1, 0.1, mem_cell_ct, concat_len)

        # bias terms
        self.bg = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bi = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bf = rand_arr(-0.1, 0.1, mem_cell_ct) 
        self.bo = rand_arr(-0.1, 0.1, mem_cell_ct) 

        # diffs (derivative of loss function w.r.t. all parameters)
        self.wg_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wi_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wf_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.wo_diff = np.zeros((mem_cell_ct, concat_len)) 
        self.bg_diff = np.zeros(mem_cell_ct) 
        self.bi_diff = np.zeros(mem_cell_ct) 
        self.bf_diff = np.zeros(mem_cell_ct) 
        self.bo_diff = np.zeros(mem_cell_ct) 

    def apply_diff(self, lr = 1):
        self.wg -= lr * self.wg_diff
        self.wi -= lr * self.wi_diff
        self.wf -= lr * self.wf_diff
        self.wo -= lr * self.wo_diff
        self.bg -= lr * self.bg_diff
        self.bi -= lr * self.bi_diff
        self.bf -= lr * self.bf_diff
        self.bo -= lr * self.bo_diff
        # reset diffs to zero
        self.wg_diff = np.zeros_like(self.wg)
        self.wi_diff = np.zeros_like(self.wi) 
        self.wf_diff = np.zeros_like(self.wf) 
        self.wo_diff = np.zeros_like(self.wo) 
        self.bg_diff = np.zeros_like(self.bg)
        self.bi_diff = np.zeros_like(self.bi) 
        self.bf_diff = np.zeros_like(self.bf) 
        self.bo_diff = np.zeros_like(self.bo) 

class LstmState:
    """
    Stores a single state in the LSTM (i.e., one time step)
    """
    def __init__(self, mem_cell_ct, x_dim):
        self.g = np.zeros(mem_cell_ct)
        self.i = np.zeros(mem_cell_ct)
        self.f = np.zeros(mem_cell_ct)
        self.o = np.zeros(mem_cell_ct)
        self.s = np.zeros(mem_cell_ct)
        self.h = np.zeros(mem_cell_ct)
        self.bottom_diff_h = np.zeros_like(self.h)
        self.bottom_diff_s = np.zeros_like(self.s)
    
class LstmNode:
    """
    A state plus its parameters.
    To represent an "unrolled" LSTM.
    """
    def __init__(self, lstm_param, lstm_state):
        # store reference to parameters and to activations
        self.state = lstm_state
        self.param = lstm_param
        # non-recurrent input concatenated with recurrent input
        self.xc = None

    def bottom_data_is(self, x, s_prev = None, h_prev = None):
        # if this is the first lstm node in the network
        if s_prev is None: s_prev = np.zeros_like(self.state.s)
        if h_prev is None: h_prev = np.zeros_like(self.state.h)
        # save data for use in backprop
        self.s_prev = s_prev
        self.h_prev = h_prev

        # concatenate x(t) and h(t-1)
        xc = np.hstack((x,  h_prev))
        self.state.g = np.tanh(np.dot(self.param.wg, xc) + self.param.bg)
        self.state.i = sigmoid(np.dot(self.param.wi, xc) + self.param.bi)
        self.state.f = sigmoid(np.dot(self.param.wf, xc) + self.param.bf)
        self.state.o = sigmoid(np.dot(self.param.wo, xc) + self.param.bo)
        self.state.s = self.state.g * self.state.i + s_prev * self.state.f
        self.state.h = self.state.s * self.state.o

        self.xc = xc
    
    def top_diff_is(self, top_diff_h, top_diff_s):
        # notice that top_diff_s is carried along the constant error carousel
        ds = self.state.o * top_diff_h + top_diff_s
        do = self.state.s * top_diff_h
        di = self.state.g * ds
        dg = self.state.i * ds
        df = self.s_prev * ds

        # diffs w.r.t. vector inside sigma / tanh function
        di_input = sigmoid_derivative(self.state.i) * di 
        df_input = sigmoid_derivative(self.state.f) * df 
        do_input = sigmoid_derivative(self.state.o) * do 
        dg_input = tanh_derivative(self.state.g) * dg

        # diffs w.r.t. inputs
        self.param.wi_diff += np.outer(di_input, self.xc)
        self.param.wf_diff += np.outer(df_input, self.xc)
        self.param.wo_diff += np.outer(do_input, self.xc)
        self.param.wg_diff += np.outer(dg_input, self.xc)
        self.param.bi_diff += di_input
        self.param.bf_diff += df_input       
        self.param.bo_diff += do_input
        self.param.bg_diff += dg_input       

        # compute bottom diff
        dxc = np.zeros_like(self.xc)
        dxc += np.dot(self.param.wi.T, di_input)
        dxc += np.dot(self.param.wf.T, df_input)
        dxc += np.dot(self.param.wo.T, do_input)
        dxc += np.dot(self.param.wg.T, dg_input)

        # save bottom diffs
        self.state.bottom_diff_s = ds * self.state.f
        self.state.bottom_diff_h = dxc[self.param.x_dim:]

class LstmNetwork():
    def __init__(self, lstm_param):
        self.lstm_param = lstm_param
        self.lstm_node_list = []
        # input sequence
        self.x_list = []

    def y_list_is(self, y_list, loss_layer):
        """
        Updates diffs by setting target sequence 
        with corresponding loss layer. 
        Will *NOT* update parameters.  To update parameters,
        call self.lstm_param.apply_diff()
        """
        assert len(y_list) == len(self.x_list)
        idx = len(self.x_list) - 1
        # first node only gets diffs from label ...
        loss = loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
        diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
        # here s is not affecting loss due to h(t+1), hence we set equal to zero
        diff_s = np.zeros(self.lstm_param.mem_cell_ct)
        self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
        idx -= 1

        ### ... following nodes also get diffs from next nodes, hence we add diffs to diff_h
        ### we also propagate error along constant error carousel using diff_s
        while idx >= 0:
            loss += loss_layer.loss(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h = loss_layer.bottom_diff(self.lstm_node_list[idx].state.h, y_list[idx])
            diff_h += self.lstm_node_list[idx + 1].state.bottom_diff_h
            diff_s = self.lstm_node_list[idx + 1].state.bottom_diff_s
            self.lstm_node_list[idx].top_diff_is(diff_h, diff_s)
            idx -= 1 

        return loss

    def x_list_clear(self):
        self.x_list = []

    def x_list_add(self, x):
        self.x_list.append(x)
        if len(self.x_list) > len(self.lstm_node_list):
            # need to add new lstm node, create new state mem
            lstm_state = LstmState(self.lstm_param.mem_cell_ct, self.lstm_param.x_dim)
            self.lstm_node_list.append(LstmNode(self.lstm_param, lstm_state))

        # get index of most recent x input
        idx = len(self.x_list) - 1
        if idx == 0:
            # no recurrent inputs yet
            self.lstm_node_list[idx].bottom_data_is(x)
        else:
            s_prev = self.lstm_node_list[idx - 1].state.s
            h_prev = self.lstm_node_list[idx - 1].state.h
            self.lstm_node_list[idx].bottom_data_is(x, s_prev, h_prev)

In [53]:
"""
Test the LSTM by learning a function from
a random vector of length 10 to a real number.

Train on a single sequence of four elements.

"""
import numpy as np
class ToyLossLayer:
    """
    Computes square loss with first element of hidden layer array.
    """
    @classmethod
    def loss(self, pred, label):
        return (pred[0] - label) ** 2

    @classmethod
    def bottom_diff(self, pred, label):
        diff = np.zeros_like(pred)
        diff[0] = 2 * (pred[0] - label)
        return diff

def example_0():
    # learns to repeat simple sequence from random inputs
    np.random.seed(0)

    # parameters for input data dimension and lstm cell count 
    mem_cell_ct = 100
    x_dim = 10
    lstm_param = LstmParam(mem_cell_ct, x_dim) 
    lstm_net = LstmNetwork(lstm_param)
    # true outputs
    y_list = [-0.5, 0.2, 0.1, -0.5]
    # input vectors (10 numbers per time step)
    input_val_arr = [np.random.random(x_dim) for _ in y_list]

    for cur_iter in range(100):
        # set inputs for this iteration.
        for ind in range(len(y_list)):
            lstm_net.x_list_add(input_val_arr[ind])

        # compute loss across all 4 inputs
        loss = lstm_net.y_list_is(y_list, ToyLossLayer)
        if cur_iter % 10 == 0:  # debugging info
            print("\ncur iter: " + str(cur_iter))
            print("loss: " + str(loss))
            for ind in range(len(y_list)):
                print("y_pred[" + str(ind) + "] : " + str(lstm_net.lstm_node_list[ind].state.h[0]))

        # update parameters
        lstm_param.apply_diff(lr=0.1)
        lstm_net.x_list_clear()
    return lstm_net, input_val_arr

lstm_net, input_val_arr = example_0()


cur iter: 0
loss: 0.661219107965
y_pred[0] : 0.027674158345
y_pred[1] : 0.0725366189921
y_pred[2] : 0.0962898251627
y_pred[3] : 0.10540764092

cur iter: 10
loss: 0.426647363868
y_pred[0] : -0.115508299425
y_pred[1] : -0.12635068764
y_pred[2] : -0.142879566421
y_pred[3] : -0.163372251666

cur iter: 20
loss: 0.378771783017
y_pred[0] : -0.145855966804
y_pred[1] : -0.105110688849
y_pred[2] : -0.130119057315
y_pred[3] : -0.172423331449

cur iter: 30
loss: 0.315378982299
y_pred[0] : -0.1893560152
y_pred[1] : -0.0739509795628
y_pred[2] : -0.114322344216
y_pred[3] : -0.18711649041

cur iter: 40
loss: 0.235246632085
y_pred[0] : -0.248195670373
y_pred[1] : -0.0276285046053
y_pred[2] : -0.0948942701423
y_pred[3] : -0.21356903271

cur iter: 50
loss: 0.152863775252
y_pred[0] : -0.314301831593
y_pred[1] : 0.0320871163779
y_pred[2] : -0.0721680004187
y_pred[3] : -0.253944297672

cur iter: 60
loss: 0.0888274951271
y_pred[0] : -0.373542300708
y_pred[1] : 0.0916656402332
y_pred[2] : -0.0468066241615
y_

In [54]:
# example of the input vectors.
x_dim = 10
input_val_arr

[array([ 0.67781654,  0.27000797,  0.73519402,  0.96218855,  0.24875314,
         0.57615733,  0.59204193,  0.57225191,  0.22308163,  0.95274901]),
 array([ 0.44712538,  0.84640867,  0.69947928,  0.29743695,  0.81379782,
         0.39650574,  0.8811032 ,  0.58127287,  0.88173536,  0.69253159]),
 array([ 0.72525428,  0.50132438,  0.95608363,  0.6439902 ,  0.42385505,
         0.60639321,  0.0191932 ,  0.30157482,  0.66017354,  0.29007761]),
 array([ 0.61801543,  0.4287687 ,  0.13547406,  0.29828233,  0.56996491,
         0.59087276,  0.57432525,  0.65320082,  0.65210327,  0.43141844])]

In [44]:
# This LSTM just outputs the first
# element of it's hidden layer as the prediction
y_list = [-0.5, 0.2, 0.1, -0.5]  # truth
lstm_net.x_list_clear()
lstm_net.x_list_add(input_val_arr[0])
lstm_net.x_list_add(input_val_arr[1])
lstm_net.x_list_add(input_val_arr[2])
lstm_net.x_list_add(input_val_arr[3])
print('%d LSTM nodes created' % len(lstm_net.lstm_node_list))
for i in range(4):
    print('prediction for input %d is %g' % (i, lstm_net.lstm_node_list[i].state.h[0]))

4 LSTM nodes created
prediction for input 0 is -0.482451
prediction for input 1 is 0.189175
prediction for input 2 is 0.0458517
prediction for input 3 is -0.42936


## Keras

There are many good libraries out there for deep learning.

- Keras (https://keras.io/) provides a wrapper over Theano and TensorFlow, two of the most popular libraries.

- See examples here:
https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

#### sources
- https://arxiv.org/abs/1506.00019
- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://github.com/nicodjimenez/lstm
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [1]:
from IPython.core.display import HTML
HTML(open('../custom.css').read())